EXP 6-Recognize traffic sign using ML algorithm

In [17]:
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def load_and_prepare_image(image_path):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Resize the image to a fixed size
    image = cv2.resize(image, (100, 100))
    # Flatten the image to a 1D array of pixel values
    return image.flatten()

def train_classifier(sign_paths):
    # Prepare data for training
    data = []
    labels = []
    
    for label, path in sign_paths.items():
        # Load each sign image and flatten
        features = load_and_prepare_image(path)
        data.append(features)
        labels.append(label)
    
    # Encode labels
    le = LabelEncoder()
    labels_encoded = le.fit_transform(labels)
    
    # Initialize and train the Random Forest classifier
    classifier = RandomForestClassifier()
    classifier.fit(data, labels_encoded)
    
    return classifier, le

def recognize_traffic_sign(input_image_path, classifier, label_encoder):
    # Prepare the input image
    input_features = load_and_prepare_image(input_image_path).reshape(1, -1)
    
    # Predict the label for the input image
    label_encoded = classifier.predict(input_features)
    
    # Decode the label back to the original label name
    recognized_sign = label_encoder.inverse_transform(label_encoded)[0]
    
    return recognized_sign

# Paths to the sign images
sign_paths = {
    'Stop Sign': "C:/Users/Admin/Downloads/ipcv/1/stop.png",
    'right Sign': "C:/Users/Admin/Downloads/ipcv/3/right.jpeg",
    'Speed Limit Sign': "C:/Users/Admin/Downloads/ipcv/2/speed.jpeg",
    'No Entry Sign': "C:/Users/Admin/Downloads/ipcv/4/entry.jpeg"
}
input_image_path = "C:/Users/Admin/Downloads/ipcv/2/speed.jpeg"

# Train the classifier
classifier, label_encoder = train_classifier(sign_paths)

# Run the recognition
recognized_sign = recognize_traffic_sign(input_image_path, classifier, label_encoder)

if recognized_sign:
    print(f"Recognized Traffic Sign: {recognized_sign}")
else:
    print("No matching traffic sign found.")


Recognized Traffic Sign: Speed Limit Sign


EXP 9: Classify digits in MNIST dataset

In [54]:
import cv2
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load MNIST dataset and split into train/test
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape and normalize the dataset
X_train = X_train.values.reshape(-1, 28, 28, 1) / 255.0
X_test = X_test.values.reshape(-1, 28, 28, 1) / 255.0


# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train_categorical, epochs=1,  batch_size=32)

# Save the trained model
model.save('mnist_cnn_model.h5')

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_categorical)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Function to recognize digit from image
def recognize_digit(image_path):
    image = cv2.resize(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE), (28, 28))
    image = image.reshape(1, 28, 28, 1) / 255.0
    prediction = model.predict(image)
    return np.argmax(prediction)

# Example usage
input_image_path = r"C:\Users\Admin\Downloads\ipcv\seven.jpeg"
print(f"Predicted Digit: {recognize_digit(input_image_path)}")


C:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1750/1750 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.9027 - loss: 0.3061


438/438 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9830 - loss: 0.0542
Test Accuracy: 98.26%


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Predicted Digit: 7


EXP 5: Detect foreground and background in video

In [1]:
import cv2

# Initialize video capture and background subtractor
video = cv2.VideoCapture(r"C:\Users\Admin\Downloads\3399820155-preview.mp4")
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = video.read()  # Read a frame
    if not ret: break  # Exit if there are no frames left

    # Apply background subtraction
    fg_mask = bg_subtractor.apply(frame)  
    cv2.imshow('Foreground', fg_mask)  # Show foreground mask

    # Exit if the ESC key is pressed
    if cv2.waitKey(30) == 27: break  

# Clean up
video.release()
cv2.destroyAllWindows()


In [11]:
import cv2
import numpy as np

image = cv2.imread(r"C:\Users\Admin\Downloads\ipcv\circle.png")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150)

# Detect lines
lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 100)
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Detect circles
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30)
if circles is not None:
    for x, y, r in np.uint16(np.around(circles[0])):
        cv2.circle(image, (x, y), r, (255, 0, 0), 2)

# Detect ellipses
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
    if len(contour) >= 5:
        cv2.ellipse(image, cv2.fitEllipse(contour), (0, 0, 255), 2)

cv2.imshow("Detected Lines, Circles, and Ellipses", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [15]:
import cv2

# Load the image
image_path = r"C:\Users\Admin\Downloads\ipcv\counter detection.jpeg"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Perform edge detection using Canny
edges = cv2.Canny(blurred, 50, 150)
# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

contoured_image = image.copy()
cv2.drawContours(contoured_image, contours, -1, (0, 255, 0), 2)  # Green color with thickness of 2
# Show the original and contoured images
cv2.imshow('Original Image', image)
cv2.imshow('Contours', contoured_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [26]:
pip install 

Note: you may need to restart the kernel to use updated packages.
